## Install `wandb` and log in

If you don't yet have a Weights and Biases account you can create one here: https://wandb.ai/login?signup=true

In [1]:
!git clone https://github.com/PaddlePaddle/PaddleOCR
!pip install -qqq paddlepaddle-gpu pyclipper attrdict
%cd PaddleOCR
!pip install -e .

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 86509, done.
remote: Counting objects: 100% (818/818), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 86509 (delta 783), reused 730 (delta 730), pack-reused 85691 (from 2)
Receiving objects: 100% (86509/86509), 563.30 MiB | 39.19 MiB/s, done.
Resolving deltas: 100% (64981/64981), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 180.0 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
/kaggle/working/PaddleOCR
Obtaining file:///kaggle/working/PaddleOCR
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -qqq wandb

In [3]:
import wandb
wandb.login(key="d292a8737bea14e34b4892facf2f363811452d5b")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 21520055 (21520055-university-of-information-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%cd ..

/kaggle/working


In [5]:
# Download the pre-trained model of en_PP-OCRv4
!wget https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar

--2024-12-23 14:24:31--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204093440 (195M) [application/x-tar]
Saving to: ‘en_PP-OCRv3_rec_train.tar’

en_PP-OCRv3_rec_tra 100%[===================>] 194.64M  12.3MB/s    in 31s     

2024-12-23 14:25:06 (6.19 MB/s) - ‘en_PP-OCRv3_rec_train.tar’ saved [204093440/204093440]



In [6]:
!tar -xf /kaggle/working/en_PP-OCRv3_rec_train.tar

# Verify your hyperparameters: edit your `config` file

For the MobileNetv3 model the configuration file with its hyperparameters are located in the `configs/det/det_mv3_db.yml` file. Note that each model has a config file that 'matches' with its name, so if you are using a model other than MobileNetv3 make sure to use the correct YAML file with your model.

Because we're running this as a tutorial we reduce the number of epochs to something smaller and reduce the evaluation batch step to a smaller value as well. By setting `cal_metric_during_train` to True you can "Set whether to evaluate the metric during the training process. At this time, the metric of the model under the current batch is evaluated". For more settable parameters refer to the Paddle documentation: https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.4/doc/doc_en/config_en.md


Learn more about how the `wandb` can help you manage your (hyper)parameters here: https://docs.wandb.ai/guides/track/config

In [7]:
!cat /kaggle/input/license-plates-vietnamese/dataset/en_PP-OCRv3_rec.yml

Global:
  debug: false
  use_gpu: true
  epoch_num: 400
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: /kaggle/working/output/en_PP-OCRv3_rec_ft
  save_epoch_step: 200
  eval_batch_step: [0, 100, 200, 300, 400]
  cal_metric_during_train: true
  pretrained_model: /kaggle/working/en_PP-OCRv3_rec_train/best_accuracy.pdparams
  checkpoints: null
  save_inference_dir: null
  use_visualdl: false
  infer_img: /kaggle/input/license-plates-vietnamese/dataset/test/76F101980_20241221_081914.jpg
  character_dict_path: /kaggle/working/PaddleOCR/ppocr/utils/en_dict.txt
  max_text_length: &max_text_length 20
  infer_mode: false
  use_space_char: true
  distributed: true
  save_res_path: /kaggle/working/output/rec/predicts_ppocrv3.txt


Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.005
    warmup_epoch: 5
  regularizer:
    name: L2
    factor: 3.0e-05


Architecture:
  model_type: rec
  algorithm: SVTR
  Transform:
  Backbone:
    na

## Beginning the fine-tuning process

See this page for a discussion of loading the pretrained model weights. Note that you do not need to include the file extension, so we load `ResNet50_pretrained` instead of `ResNet50_pretrained.pdparams`: https://github.com/PaddlePaddle/Paddle/issues/23509#issuecomment-610340722

In [8]:
# Fine-tune the PPOCRv3 V3 model
!python3 /kaggle/working/PaddleOCR/tools/train.py -c /kaggle/input/license-plates-vietnamese/dataset/en_PP-OCRv3_rec.yml \
     -o Global.pretrained_model=/kaggle/working/en_PP-OCRv3_rec_train/best_accuracy.pdparams

wandb: Currently logged in as: 21520055 (21520055-university-of-information-technology). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/output/en_PP-OCRv3_rec_ft/wandb/run-20241223_142511-h0ervpi7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run valiant-salad-7
wandb: ⭐️ View project at https://wandb.ai/21520055-university-of-information-technology/OCR_with_Paddle
wandb: 🚀 View run at https://wandb.ai/21520055-university-of-information-technology/OCR_with_Paddle/runs/h0ervpi7
[2024/12/23 14:25:11] ppocr INFO: Architecture : 
[2024/12/23 14:25:11] ppocr INFO:     Backbone : 
[2024/12/23 14:25:11] ppocr INFO:         last_conv_stride : [1, 2]
[2024/12/23 14:25:11] ppocr INFO:         last_pool_type : avg
[2024/12/23 14:25:11] ppocr INFO:         name : MobileNetV1Enhance
[2024/12/23 14:25:11] ppocr INFO:         scale : 0.5
[2024/12/23 14:25:11] ppocr INFO:     Head : 
[2024/

In [ ]:
# # for distributed training
# !python3 -m paddle.distributed.launch --gpus '0,1' /kaggle/working/PaddleOCR/tools/train.py -c /kaggle/input/license-plates-vietnamese/dataset/en_PP-OCRv3_rec.yml \
#      -o Global.pretrained_model=/content/en_PP-OCRv3_rec_train/best_accuracy.pdparams

In [10]:
!mkdir -p /kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy
!mv /kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy.pdopt /kaggle/working//best_accuracy/
!mv /kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy.pdparams /kaggle/working/best_accuracy/
!mv /kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy.states /kaggle/working/best_accuracy/


In [23]:
!zip -r /kaggle/working/best_accuracy.zip /kaggle/working/best_accuracy
### Download best_accuracy.zip

  adding: kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy/ (stored 0%)
  adding: kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy/best_accuracy.pdopt (deflated 23%)
  adding: kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy/best_accuracy.pdparams (deflated 7%)
  adding: kaggle/working/output/en_PP-OCRv3_rec_ft/best_accuracy/best_accuracy.states (deflated 9%)
